<a href="https://colab.research.google.com/github/freud-sensei/counsel_chatbot/blob/main/%5B%ED%86%A0%EC%9D%B4%EC%8A%A4%ED%86%A0%EB%A6%AC%5D_%EC%8B%A4%EC%A0%9C_%EC%8B%AC%EB%A6%AC%EC%83%81%EB%8B%B4_%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%83%81%EB%8B%B4%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=86

In [ ]:
!unzip -qq "/content/drive/MyDrive/project/감성대화말뭉치(최종데이터)_Training.zip" -d '/content/drive/MyDrive/project'
!unzip -qq "/content/drive/MyDrive/project/감성대화말뭉치(최종데이터)_Validation.zip" -d '/content/drive/MyDrive/project'

In [ ]:
import json
with open("/content/drive/MyDrive/project/감성대화말뭉치(최종데이터)_Training.json") as f:
  js_train = json.loads(f.read())
with open("/content/drive/MyDrive/project/감성대화말뭉치(최종데이터)_Validation.json") as f:
  js_test = json.loads(f.read())

In [ ]:
import pandas as pd
import numpy as np

def preprocessing(js):
  df = pd.json_normalize(js)
  columns_to_keep = [f"talk.content.{i}" for i in ["HS01", "HS02", "HS03", "SS01", "SS02", "SS03"]]
  df = df[columns_to_keep]
  df.columns = ['HS01', 'HS02', 'HS03', 'SS01', 'SS02', 'SS03']
  result = pd.DataFrame({
      'hs': pd.concat([df['HS01'], df['HS02'], df['HS03']], ignore_index=True),
      'ss': pd.concat([df['SS01'], df['SS02'], df['SS03']], ignore_index=True)
  })
  result = result[~(result == "").any(axis=1)]
  result = result.reset_index(drop=True)
  return result

emotion_train_s = preprocessing(js_train)
emotion_test_s = preprocessing(js_test)

In [ ]:
emotion_train_s

,hs,ss
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,많이 힘드시겠어요. 주위에 의논할 상대가 있나요?
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 ...
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있...
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는...
...,...,...
145943,노년층을 위한 경제적 지원이나 부업 같은 것도 알아보아야겠어.,좋은 결과 얻으시길 바랄게요.
145944,남편과 함께 게이트볼이나 치러 가야겠어. 그럼 기분이 나아질 것 같아.,남편과 함께하는 좋은 외출 시간 되시길 바랄게요.
145945,남편과 함께 실버 일자리나 노년층을 위한 국가 지원에 대해 자세히 알아보아야겠어.,좋은 정보 많이 얻으셔서 걱정을 좀 덜으셨으면 좋겠어요.
145946,함께 친하게 지내던 동네 언니 동생들과 빈자리를 조금이나마 채울까 해.,지인분들과 좋은 시간 보내셨으면 좋겠어요.


In [ ]:
emotion_train_s.to_pickle('/content/drive/MyDrive/project/train.pkl')
emotion_test_s.to_pickle('/content/drive/MyDrive/project/test.pkl')

# 불러오기

In [ ]:
!pip install transformers[torch]
!pip install datasets
import torch

In [ ]:
from datasets import Dataset, DatasetDict
import pandas as pd
emotion_train_s = pd.read_pickle('/content/drive/MyDrive/project/train.pkl')
emotion_test_s = pd.read_pickle('/content/drive/MyDrive/project/test.pkl')

emotion_dataset_s = DatasetDict({
    "train": Dataset.from_pandas(emotion_train_s),
    "test": Dataset.from_pandas(emotion_test_s)
})

In [ ]:
emotion_dataset_s["train"][:10]

{'hs': ['일은 왜 해도 해도 끝이 없을까? 화가 난다.',
  '이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.',
  '회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스트레스 받아. ',
  '직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 섭섭해.',
  '얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.',
  '직장에 다니고 있지만 시간만 버리는 거 같아. 진지하게 진로에 대한 고민이 생겨.',
  '성인인데도 진로를 아직도 못 정했다고 부모님이 노여워하셔. 나도 섭섭해.',
  '퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.',
  '졸업반이라서 취업을 생각해야 하는데 지금 너무 느긋해서 이래도 되나 싶어.',
  '요즘 직장생활이 너무 편하고 좋은 것 같아!'],
 'ss': ['많이 힘드시겠어요. 주위에 의논할 상대가 있나요?',
  '급여가 줄어 속상하시겠어요. 월급이 줄어든 것을 어떻게 보완하실 건가요?',
  '회사 동료 때문에 스트레스를 많이 받는 것 같아요. 문제 해결을 위해 어떤 노력을 하면 좋을까요?',
  '관련 없는 심부름을 모두 하게 되어서 노여우시군요. 어떤 것이 상황을 나아질 수 있게 도움을 줄까요?',
  '무시하는 것 같은 태도에 화가 나셨군요. 상대방의 어떤 행동이 그런 감정을 유발하는 것일까요?',
  '진로에 대해서 고민하고 계시는군요. 어떤 점이 고민인가요?',
  '부모님의 노여움에 섭섭하시군요. 이런 상황을 어떻게 해결하면 좋을까요? ',
  '천천히라도 직장을 구해 보려고 하시는군요. 특별한 이유가 있으신가요?',
  '취업에 대해 걱정이 되는군요.',
  '직장생활이 편하고 좋으시다니 좋아 보여요. 다니고 계신 회사만의 장점이 있나요?']}

# 학습

무지한 중생을 구한 참고자료

https://www.kaggle.com/code/changyeop/how-to-fine-tune-gpt-2-for-beginners

https://github.com/ukairia777/tensorflow-nlp-tutorial/blob/main/22.%20Fine-tuning%20GPT-2%20(Cls%2C%20Chatbot%2C%20NLI)/22-3.%20kogpt2_chatbot_gpu.ipynb

In [ ]:
from transformers import AutoTokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import TextDataset
tokenizer = AutoTokenizer.from_pretrained("skt/kogpt2-base-v2")
tokenizer.add_special_tokens({'pad_token': '<pad>'})
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")

def preprocessing_function(example):
  prompt = [f"질문: {question} 답변:" for question in example['hs']]
  target = example['ss']
  return tokenizer(prompt, target, truncation="only_second", max_length=128)

tokenized_dataset = emotion_dataset_s.map(preprocessing_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['hs', 'ss'])

Map:   0%|          | 0/145948 [00:00<?, ? examples/s]

Map:   0%|          | 0/17965 [00:00<?, ? examples/s]

시간 관계상 epoch=1로 설정했다. 실제로 연구를 할 땐 이것보다 더 돌려야 함 물론....

In [ ]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForLanguageModeling

training_args = TrainingArguments("counsel_chatbot", num_train_epochs=1, push_to_hub=True)
trainer = Trainer(
    model=model, args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
)
trainer.train()

trainer.push_to_hub()

Step,Training Loss
500,2.302900
1000,2.879600
1500,2.816000
2000,2.790800
2500,2.745700
3000,2.732900
3500,2.702700
4000,2.684300
4500,2.666800
5000,2.644100


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub()

Colab의 한계로 여기까지만 진행합시다.

# output generation

In [ ]:
from transformers import pipeline
def chatbot(input):
  pipeline_model = pipeline("text-generation", model="freud-sensei/counsel_chatbot")
  sentence = f"질문: {input} 답변:"
  answer = pipeline_model(sentence)
  return answer[0]["generated_text"][len(sentence):]

In [ ]:
answer = chatbot("어제 잠을 거의 못 자서 피곤해.")
print(answer)

 숙면을 못 주셔서 피곤하시군요. 어떤 이유가 있나요?                     


# gradio

In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.2/305.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s e

In [ ]:
import gradio as gr
from transformers import pipeline

def counsel_response(message):
  pipeline_model = pipeline("text-generation", model="freud-sensei/counsel_chatbot")
  sentence = f"질문: {input} 답변:"
  answer = pipeline_model(sentence)
  return answer[0]["generated_text"][len(sentence):]

chatbot = gr.Interface(fn=counsel_response, inputs="text", outputs="text")
chatbot.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://5aa32b11a9085ce0cf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
def counsel_chat(message, history):
  pipeline_model = pipeline("text-generation", model="freud-sensei/counsel_chatbot")
  sentence = f"질문: {input} 답변:"
  answer = pipeline_model(sentence)
  return answer[0]["generated_text"][len(sentence):]

chatbot2 = gr.ChatInterface(counsel_chat)
chatbot2.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://58c14709afce28bc24.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
